In [1]:
import pandas as pd
import plotly.express as px

c:\Users\pcost\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [12]:
games_Medal = pd.read_csv('Dataset/Olympic_Games_Medal_Tally.csv', sep=',', encoding_errors='replace')
games_Medal = games_Medal.replace(["na"], None)

games_Medal = games_Medal[games_Medal["edition"].str.contains("Winter") == False]

df = pd.DataFrame(games_Medal["country"].unique(), columns=["country"])

total = games_Medal.groupby('country')['total'].sum()

df = pd.merge(df, total, on='country', how='left')

# CLEANING DATA

df = df[df["country"].str.contains("Mixed team")==False] # Impossivel dividir
df = df[df["country"].str.contains("Unified Team")==False] # Impossivel dividir
df = df[df["country"].str.contains("Individual Olympic Athletes")==False] # Impossivel dividir
df = df[df["country"].str.contains("Bohemia")==False]    # Eslováquia + Republica Checa
df = df[df["country"].str.contains("Australasia")==False]    # Australia + Nova Zelandia + Nova Guine + partes da Indonesia
df = df[df["country"].str.contains("Czechoslovakia")==False]    # Eslováquia + Republica Checa
df = df[df["country"].str.contains("Yugoslavia")==False] # Bosnia + Croacia + Macedonia + Montenegro + Eslovenia + Servia
df = df[df["country"].str.contains("Soviet Union")==False] # Russia + Letonia + Lituania + Estonia + Georgia + Armenia + Azerbaijao + Bielorrussia + Cazaquistao + Moldavia + Quirguistao + Tajiquistao + Turquemenistao + Ucrania + Usbequistao
df = df[df["country"].str.contains("United Arab Republic")==False]  # Egypt + Syria + Faixa de Gaza
df = df[df["country"].str.contains("West Indies Federation")==False] # Antigua + Barbados + Cayman Islands + Dominica + Grenada + Jamaica + Montserrat + St Christopher-Nevis-Anguilla + Saint Lucia	+ St Vincent and the Grenadines	+ Trinidad and Tobago + Turks and Caicos Islands	

df["country"] = df["country"].replace('Türkiye', 'Turkey')

df.loc[df['country'] == "Germany", ['total']] += df.loc[df['country'] == "West Germany", ['total']].values[0][0]
df.loc[df['country'] == "Germany", ['total']] += df.loc[df['country'] == "East Germany", ['total']].values[0][0]
df = df[df["country"].str.contains("West Germany")==False]
df = df[df["country"].str.contains("East Germany")==False]

df.loc[df['country'] == "People's Republic of China", ['total']] += df.loc[df['country'] == "Hong Kong, China", ['total']].values[0][0]
df = df[df["country"].str.contains("Hong Kong, China")==False]

df.loc[df['country'] == "Russian Federation", ['total']] += df.loc[df['country'] == "ROC", ['total']].values[0][0]
df = df[df["country"].str.contains("ROC")==False]

#df = df[df["total"] > 5]


olympic_Country = pd.read_csv('Dataset/Olympics_Country.csv', sep=',')
olympic_Country = olympic_Country.replace(["na"], None)

df = pd.merge(df, olympic_Country, on='country', how='left')

display(df)

    country_noc                    country
0           AFG                Afghanistan
1           ALB                    Albania
2           ALG                    Algeria
3           ASA             American Samoa
4           AND                    Andorra
..          ...                        ...
229         YEM                      Yemen
230         YUG                 Yugoslavia
231         ZAM                     Zambia
232         ZIM                   Zimbabwe
233         ROC  Russian Olympic Committee

[234 rows x 2 columns]


,country,total,country_noc
0,United States,2655,USA
1,Greece,122,GRE
2,Germany,1404,GER
3,France,772,FRA
4,Great Britain,928,GBR
...,...,...,...
134,Jordan,3,JOR
135,Kosovo,3,KOS
136,San Marino,3,SMR
137,Turkmenistan,1,TKM


In [38]:
top10 = df.sort_values(by=['total'], ascending=False).head(10).reset_index(drop=True)

print(top10)

fig = px.bar(top10, x="country", y="total", title="Top 10 Countries by Total Medals", width=400, height=800)
fig.update_layout(font_family= 'Cabin',autosize = False, 
                      legend=dict(yanchor="top", xanchor="left", font=dict(size=15)),
                      margin=dict(l=1, r=6, b=20, t=31, pad=0))
fig.show()

                      country  total country_noc
0               United States   2655         USA
1                     Germany   1404         GER
2               Great Britain    928         GBR
3                      France    772         FRA
4  People's Republic of China    643         CHN
5                       Italy    629         ITA
6                   Australia    541         AUS
7                     Hungary    515         HUN
8                      Sweden    504         SWE
9          Russian Federation    503         RUS


In [76]:
athlete_Event_Results = pd.read_csv('Dataset/Olympic_Athlete_Event_Results.csv', sep=',')
athlete_Event_Results = athlete_Event_Results.replace(["na"], None)


athlete_results = athlete_Event_Results[athlete_Event_Results["edition"].str.contains("Winter") == False]   # apenas jogos de verao

athlete_results = athlete_results.reset_index()

athlete_results = athlete_results.drop(columns=['result_id','athlete','athlete_id','pos', 'index'])

athlete_results['edition'] = athlete_results['edition'].str.split().str[0].astype(int)


repeat = []
i=0
while(i < len(athlete_results)):
    if athlete_results.loc[i, 'isTeamSport'] == True:
        auxi = i + 1 
        p = athlete_results.loc[i, 'country_noc']
        e = athlete_results.loc[i, 'event']
        a = athlete_results.loc[i, 'edition_id']
        while((athlete_results.loc[auxi, 'country_noc'] == p) and (athlete_results.loc[auxi, 'event'] == e) and (athlete_results.loc[auxi, 'edition_id'] == a)):
            repeat.append(auxi)
            auxi += 1
        i = auxi-1
    i+=1

athlete_results.drop(repeat, inplace=True)

athlete_aux = athlete_results.copy()

athlete_results['medal'] = athlete_results['medal'].replace([None], 0)
athlete_results['medal'] = athlete_results['medal'].replace(['Gold'], 1)
athlete_results['medal'] = athlete_results['medal'].replace(['Silver'], 1)
athlete_results['medal'] = athlete_results['medal'].replace(['Bronze'], 1)

athlete_results = athlete_results.drop(columns=['edition_id','sport','event','isTeamSport'])

medals = pd.DataFrame(athlete_results.groupby(['country_noc']).sum()).reset_index().sort_values(by=['medal'], ascending=False).head(10)

athlete_aux = athlete_aux[athlete_aux['country_noc'].isin(medals['country_noc'])]

medals = pd.DataFrame(athlete_aux.groupby(['country_noc' , 'medal']).count()).reset_index()
medals.drop(columns=['edition_id','sport','event','isTeamSport'], inplace=True)
medals.rename(columns={'edition': 'count', 'country_noc': 'country'}, inplace=True)

colors = ['#D6AF36', '#A7A7AD', '#A77044']

fig = px.bar(medals, x="country", y="count", color='medal', 
             color_discrete_map={
                'Gold': '#D6AF36',
                'Silver': '#A7A7AD',
                'Bronze': '#A77044'}, 
            title="Top Medals by Country", width=400, height=800).update_xaxes(categoryorder="total descending").update_layout(font_family= 'Cabin')
fig.show()
